# Completeness and Post's Theorem

This notebook explains completeness in logic and Post's Completeness Theorem, which provides an efficient way to check if a set of connectives is complete.

## Learning Objectives

By the end of this notebook, you will understand:
- What completeness means in logic
- The 5 Post classes (T₀, T₁, M, D, A)
- Post's Completeness Theorem
- How to check completeness programmatically
- Famous complete sets like {NAND} and {AND, OR, NOT}

In [1]:
# Setup Python path to find the src module
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✓ Project root added to path: {project_root}")

✓ Project root added to path: /home/benjamin/Documents/Philosophy/Projects/Z3/nice_connectives


## 0. Python Path Setup

First, let's ensure Python can find the project modules:

In [2]:
# Setup
from src.connectives import Connective
from src.constants import AND, OR, XOR, NOT, NAND, TRUE, FALSE
from src.post_classes import is_complete, get_post_class_memberships
from notebooks.utils import display_truth_table, compare_connectives

import matplotlib.pyplot as plt
%matplotlib inline

print("✓ Setup complete")

✓ Setup complete


## 1. What is Completeness?

A set of connectives is **complete** if every possible logical connective can be **defined** (composed) using only the connectives in that set.

### Example: {AND, OR, NOT}

This is a famous complete set. For example:
- **XOR** can be defined: `XOR(x,y) = OR(AND(x,NOT(y)), AND(NOT(x),y))`
- **NAND** can be defined: `NAND(x,y) = NOT(AND(x,y))`

In fact, *every* binary connective can be expressed using just AND, OR, and NOT.

## 2. The Problem with Direct Checking

Checking completeness by trying all possible compositions would take **exponential time**.

Fortunately, **Post's Completeness Theorem** provides an efficient O(n) check!

## 3. Post's Five Classes

Post identified 5 special classes of connectives:

### T₀: Truth-Preserving
All inputs 0 → output 0

### T₁: Falsity-Preserving  
All inputs 1 → output 1

### M: Monotone
More 1's in input → never decreases output

### D: Dual
Self-dual under negation of inputs and output

### A: Affine
Output is XOR of inputs (linear over GF(2))

## 4. Post's Completeness Theorem

**A set S is complete if and only if S escapes all 5 Post classes.**

That is:
- At least one connective NOT in T₀
- At least one connective NOT in T₁
- At least one connective NOT in M
- At least one connective NOT in D
- At least one connective NOT in A

This reduces completeness checking from exponential to **O(n)** time!

## 5. Checking Post Class Membership

Let's check which Post classes some common connectives belong to:

In [3]:
# Check Post class memberships
connectives = [AND, OR, XOR, NOT, NAND]

print("Post Class Memberships:")
print("Connective | T₀  T₁  M   D   A")
print("-----------+--------------------")

for conn in connectives:
    memberships = get_post_class_memberships(conn)
    t0 = "✓" if 'T0' in memberships else "✗"
    t1 = "✓" if 'T1' in memberships else "✗"
    m = "✓" if 'M' in memberships else "✗"
    d = "✓" if 'D' in memberships else "✗"
    a = "✓" if 'A' in memberships else "✗"
    print(f"{conn.name:10} | {t0}   {t1}   {m}   {d}   {a}")

Post Class Memberships:
Connective | T₀  T₁  M   D   A
-----------+--------------------
AND        | ✓   ✓   ✓   ✗   ✗
OR         | ✓   ✓   ✓   ✗   ✗
XOR        | ✓   ✗   ✗   ✗   ✓
NOT        | ✗   ✗   ✗   ✓   ✓
NAND       | ✗   ✗   ✗   ✗   ✗


### Understanding the Results

Let's examine why AND is in T₀:

In [4]:
# Check T₀ property for AND
print("AND with all inputs 0:")
print(f"AND(0,0) = {AND.evaluate(0,0)}")
print()
print("AND is in T₀ because AND(0,0) = 0 (truth-preserving)")
print()
print("Is AND in T₁?")
print(f"AND(1,1) = {AND.evaluate(1,1)}")
print("Yes! AND is also in T₁.")

AND with all inputs 0:
AND(0,0) = 0

AND is in T₀ because AND(0,0) = 0 (truth-preserving)

Is AND in T₁?
AND(1,1) = 1
Yes! AND is also in T₁.


Why is NOT not in M (monotone)?

In [5]:
# Check monotonicity for NOT
print("NOT violates monotonicity:")
print(f"NOT(0) = {NOT.evaluate(0)}")
print(f"NOT(1) = {NOT.evaluate(1)}")
print()
print("As input increases (0→1), output decreases (1→0).")
print("Therefore NOT is NOT monotone.")

NOT violates monotonicity:
NOT(0) = 1
NOT(1) = 0

As input increases (0→1), output decreases (1→0).
Therefore NOT is NOT monotone.


## 6. Checking Completeness: {AND, OR, NOT}

Let's verify that {AND, OR, NOT} is complete:

In [6]:
# Check completeness of {AND, OR, NOT}
set1 = [AND, OR, NOT]

result = is_complete(set1)
print(f"Is {{AND, OR, NOT}} complete? {result}")
print()

# Show why it's complete
print("Why it's complete:")
print("  - AND is in T₀, but NOT escapes T₀")
print("  - AND is in T₁, but NOT escapes T₁")
print("  - AND, OR are monotone, but NOT escapes M")
print("  - XOR is in A, but AND escapes A")
print("  - AND is in D? Let's check...")
print(f"    AND in D: {'D' in get_post_class_memberships(AND)}")
print(f"    OR in D: {'D' in get_post_class_memberships(OR)}")
print("  - One of them escapes D")
print()
print("All 5 classes escaped → Complete!")

Is {AND, OR, NOT} complete? True

Why it's complete:
  - AND is in T₀, but NOT escapes T₀
  - AND is in T₁, but NOT escapes T₁
  - AND, OR are monotone, but NOT escapes M
  - XOR is in A, but AND escapes A
  - AND is in D? Let's check...
    AND in D: False
    OR in D: False
  - One of them escapes D

All 5 classes escaped → Complete!


## 7. Famous Complete Set: {NAND}

Remarkably, **NAND alone** is complete!

In [7]:
# Check completeness of {NAND}
set2 = [NAND]

result = is_complete(set2)
print(f"Is {{NAND}} complete? {result}")
print()

# Show Post class memberships
memberships = get_post_class_memberships(NAND)
print("NAND Post class memberships:")
for class_name in ['T0', 'T1', 'M', 'D', 'A']:
    is_member = class_name in memberships
    status = "✓ IN" if is_member else "✗ ESCAPES"
    print(f"  {class_name}: {status}")
print()
print("NAND escapes all 5 classes → Complete!")

Is {NAND} complete? True

NAND Post class memberships:
  T0: ✗ ESCAPES
  T1: ✗ ESCAPES
  M: ✗ ESCAPES
  D: ✗ ESCAPES
  A: ✗ ESCAPES

NAND escapes all 5 classes → Complete!


### Why NAND is Universal

You can build any connective from NAND:

In [8]:
# Build NOT from NAND
def NOT_from_NAND(x):
    return NAND.evaluate(x, x)

# Build AND from NAND
def AND_from_NAND(x, y):
    return NAND.evaluate(NAND.evaluate(x, y), NAND.evaluate(x, y))

# Verify
print("Building connectives from NAND:")
print()
print("NOT from NAND:")
for x in [0, 1]:
    result = NOT_from_NAND(x)
    expected = NOT.evaluate(x)
    print(f"  NOT({x}) = {expected}, NAND({x},{x}) = {result} {'✓' if result == expected else '✗'}")

print()
print("AND from NAND:")
for x in [0, 1]:
    for y in [0, 1]:
        result = AND_from_NAND(x, y)
        expected = AND.evaluate(x, y)
        print(f"  AND({x},{y}) = {expected}, built from NAND = {result} {'✓' if result == expected else '✗'}")

Building connectives from NAND:

NOT from NAND:
  NOT(0) = 1, NAND(0,0) = 1 ✓
  NOT(1) = 0, NAND(1,1) = 0 ✓

AND from NAND:
  AND(0,0) = 0, built from NAND = 0 ✓
  AND(0,1) = 0, built from NAND = 0 ✓
  AND(1,0) = 0, built from NAND = 0 ✓
  AND(1,1) = 1, built from NAND = 1 ✓


## 8. Incomplete Sets

Let's see an incomplete set:

In [9]:
# Check {AND, OR} without NOT
set3 = [AND, OR]

result = is_complete(set3)
print(f"Is {{AND, OR}} complete? {result}")
print()

# Both are monotone
print("Why it's incomplete:")
print(f"  AND in M: {'M' in get_post_class_memberships(AND)}")
print(f"  OR in M: {'M' in get_post_class_memberships(OR)}")
print()
print("Both are monotone → cannot escape M → incomplete!")
print("This means: cannot define NOT or any non-monotone connective.")

Is {AND, OR} complete? False

Why it's incomplete:
  AND in M: True
  OR in M: True

Both are monotone → cannot escape M → incomplete!
This means: cannot define NOT or any non-monotone connective.


## 9. Interactive Exercise

Is {XOR, AND} complete?

In [10]:
# Check {XOR, AND}
set4 = [XOR, AND]

result = is_complete(set4)
print(f"Is {{XOR, AND}} complete? {result}")
print()

# Analyze
print("Post class memberships:")
for conn in set4:
    memberships = get_post_class_memberships(conn)
    print(f"\n{conn.name}:")
    for class_name in ['T0', 'T1', 'M', 'D', 'A']:
        is_member = class_name in memberships
        status = "IN" if is_member else "escapes"
        print(f"  {class_name}: {status}")

Is {XOR, AND} complete? False

Post class memberships:

XOR:
  T0: IN
  T1: escapes
  M: escapes
  D: escapes
  A: IN

AND:
  T0: IN
  T1: IN
  M: IN
  D: escapes
  A: escapes


## 10. Building Complete Sets

To build a complete set, you need connectives that collectively escape all 5 classes.

Let's try {XOR, AND, TRUE}:

In [11]:
# Check {XOR, AND, TRUE}
set5 = [XOR, AND, TRUE]

result = is_complete(set5)
print(f"Is {{XOR, AND, TRUE}} complete? {result}")
print()

if result:
    print("Yes! This is a nice set of size 3.")
    print("(assuming it's also independent - see next notebook)")
else:
    print("Not complete. Need different connectives.")

Is {XOR, AND, TRUE} complete? True

Yes! This is a nice set of size 3.
(assuming it's also independent - see next notebook)


## Summary

In this notebook, you learned:
- ✓ What completeness means
- ✓ The 5 Post classes (T₀, T₁, M, D, A)
- ✓ Post's Completeness Theorem
- ✓ How to check completeness in O(n) time
- ✓ Famous complete sets: {NAND}, {AND, OR, NOT}

## Key Insight

**Completeness is necessary but not sufficient for a "nice" set.**

A nice set must also be **independent** - no connective is definable from the others.

## Next Steps

- **04_independence.ipynb** - Learn about independence checking